In [1]:
import itertools
from parlay_system import *
pd.set_option('display.max_rows', 500)


In [2]:
# Dec 15 games

# NOTE: Odds boosted to +105 for a patriots, chiefs, sf49ers outcome

# line 10.5
patriots = MoneyLine(event="patriots", bet_amount=100, odds=-500)
bengals = MoneyLine(event="bengals", bet_amount=100, odds=420)

# line 9.5
broncos = MoneyLine(event="broncos", bet_amount=100, odds=370)
chiefs = MoneyLine(event="chiefs", bet_amount=100, odds=-435)

# line 10
falcons = MoneyLine(event="falcons", bet_amount=100, odds=360)
sf49ers = MoneyLine(event="sf49ers", bet_amount=100, odds=-435)

binaries = [
    [patriots, bengals],
    [broncos, chiefs],
    [falcons, sf49ers],
]

results = [
    [1, 0], #[patriots, bengals],
    [0, 1], #[broncos, chiefs],
    [0, 1], #[falcons, sf49ers],
]


def create_dicts(in_binaries, in_results):
    ml_lookup = {}
    ml_win_lookup = {}
    id = 0
    for bi in range(len(in_binaries)):
        binary = in_binaries[bi]
        for ml_i in range(len(binary)):
            ml = binary[ml_i]
            ml.set_index(id) 
            is_winner = in_results[bi][ml_i]
            ml_lookup[id] = ml.event
            ml_win_lookup[id] = is_winner
            id += 1
    
    return ml_lookup, ml_win_lookup

ml_lookup, ml_outcome_lookup = create_dicts(binaries, results)



In [3]:
#ml_lookup

In [4]:
#ml_outcome_lookup

In [5]:
def findsubsets(s, n):
    return list(itertools.combinations(s, n))

TOURNEY_SIZE = len(binaries)
SELECT_NUM = 3
s = set()
n = SELECT_NUM

for i in range(TOURNEY_SIZE):
    s.add(i)

all_subsets = findsubsets(s, n)

In [6]:
len(all_subsets)

1

In [7]:
csv = []
df = pd.DataFrame()

for i in range(len(all_subsets)):
    current_subset = all_subsets[i]
    parlay_binaries = []
    for i in range(len(current_subset)):
        index = current_subset[i]
        parlay_binaries.append(binaries[index])

    ps = ParlaySystem(binaries=parlay_binaries,
                    target_profit=0.50,
                    bounds=(0.01, 30),
                    binary_index_arr=list(current_subset),
                    binary_results_arr=results,
                    index_to_ml=ml_lookup,
                    index_to_outcome=ml_outcome_lookup
                    )
    csv_res, raw_df = ps.slsqp_solver()
    df = pd.concat([df, raw_df])
    csv.append(csv_res)

     fun: -0.39684405809422885
     jac: array([-2.36800000e+00,  7.93103501e-03,  2.63793096e-02,  6.48107409e-01,
       -1.31780000e+01, -2.88229885e+00, -2.80235632e+00, -1.08201217e-01])
 message: 'Positive directional derivative for linesearch'
    nfev: 70
     nit: 9
    njev: 5
  status: 8
 success: False
       x: array([0.03595465, 0.10363774, 0.10291628, 0.01      , 0.01      ,
       0.03103318, 0.03170781, 0.1075577 ])
slsqp_solver: 
                       event  event_status   bet        mult  payout  profit
3   patriots_chiefs_sf49ers_          True  0.01    1.815141  0.0182   -0.41
4   bengals_broncos_falcons_         False  0.01  112.424000  1.1242    0.69
0  patriots_broncos_falcons_         False  0.04   25.944000  0.9328    0.50
5   bengals_broncos_sf49ers_         False  0.03   30.058391  0.9328    0.50
6    bengals_chiefs_falcons_         False  0.03   29.418851  0.9328    0.50
7    bengals_chiefs_sf49ers_         False  0.11    7.865610  0.8460    0.41
1  patrio

In [8]:
df.describe()

,bet,mult,payout,profit
count,8.000000,8.000000,8.000000,8.00000
mean,0.053750,27.656439,0.775550,0.34375
std,0.042405,36.098204,0.334535,0.33252
min,0.010000,1.815141,0.018200,-0.41000
25%,0.025000,6.899655,0.713850,0.28500
50%,0.035000,16.904805,0.889400,0.45500
75%,0.100000,29.578736,0.932800,0.50000
max,0.110000,112.424000,1.124200,0.69000


In [9]:
negative_df = df.loc[df.profit < 0]
negative_df = negative_df.sort_values(by=['event_status', 'profit'], ascending=[False, True])
negative_df.describe()

,bet,mult,payout,profit
count,1.00,1.000000,1.0000,1.00
mean,0.01,1.815141,0.0182,-0.41
std,NaN,NaN,NaN,NaN
min,0.01,1.815141,0.0182,-0.41
25%,0.01,1.815141,0.0182,-0.41
50%,0.01,1.815141,0.0182,-0.41
75%,0.01,1.815141,0.0182,-0.41
max,0.01,1.815141,0.0182,-0.41


In [10]:
positive_df = df.loc[df.profit >= 0]
positive_df = positive_df.sort_values(by=['event_status', 'profit'], ascending=[False, False])
positive_df.describe()

,bet,mult,payout,profit
count,7.000000,7.000000,7.000000,7.000000
mean,0.060000,31.348053,0.883743,0.451429
std,0.041633,37.323793,0.145990,0.144156
min,0.010000,6.788966,0.698700,0.270000
25%,0.030000,7.401081,0.782450,0.350000
50%,0.040000,25.944000,0.932800,0.500000
75%,0.100000,29.738621,0.932800,0.500000
max,0.110000,112.424000,1.124200,0.690000


In [11]:
negative_df.head(200)

,event,event_status,bet,mult,payout,profit
3,patriots_chiefs_sf49ers_,True,0.01,1.815141,0.0182,-0.41


In [12]:
positive_df.head(200)

,event,event_status,bet,mult,payout,profit
4,bengals_broncos_falcons_,False,0.01,112.424000,1.1242,0.69
0,patriots_broncos_falcons_,False,0.04,25.944000,0.9328,0.50
5,bengals_broncos_sf49ers_,False,0.03,30.058391,0.9328,0.50
6,bengals_chiefs_falcons_,False,0.03,29.418851,0.9328,0.50
7,bengals_chiefs_sf49ers_,False,0.11,7.865610,0.8460,0.41
1,patriots_broncos_sf49ers_,False,0.10,6.936552,0.7189,0.29
2,patriots_chiefs_falcons_,False,0.10,6.788966,0.6987,0.27


In [13]:
positive_df.groupby('event_status').count()

,event,bet,mult,payout,profit
event_status,,,,,
False,7,7,7,7,7


In [14]:
negative_df.groupby('event_status').count()

,event,bet,mult,payout,profit
event_status,,,,,
True,1,1,1,1,1


In [15]:
neg = negative_df.reset_index(drop=True).style.applymap(color_negative_red)
neg

,event,event_status,bet,mult,payout,profit
0,patriots_chiefs_sf49ers_,True,0.01,1.81514,0.0182,-0.41


In [16]:
pos = positive_df.reset_index(drop=True).style.applymap(color_positive_green)
pos

,event,event_status,bet,mult,payout,profit
0,bengals_broncos_falcons_,False,0.01,112.424,1.1242,0.69
1,patriots_broncos_falcons_,False,0.04,25.944,0.9328,0.5
2,bengals_broncos_sf49ers_,False,0.03,30.0584,0.9328,0.5
3,bengals_chiefs_falcons_,False,0.03,29.4189,0.9328,0.5
4,bengals_chiefs_sf49ers_,False,0.11,7.86561,0.846,0.41
5,patriots_broncos_sf49ers_,False,0.1,6.93655,0.7189,0.29
6,patriots_chiefs_falcons_,False,0.1,6.78897,0.6987,0.27


In [18]:
sf49ers = MoneyLine(event="parlay", bet_amount=0.01, odds=110)
sf49ers.print_stats()

event:  parlay
bet_amount:  0.01
odds:  110
multiplier:  2.1
payout: 0.021
